In [2]:
import pandas as pd

# Wczytanie konkretnego arkusza
df_table = pd.read_excel("../data/raw/Dane_Ekstraklasa.xlsx",
                         sheet_name="Tabela_ligi_24_25",
                         engine="openpyxl")


print(df_table.head())
print(df_table.columns)
df_table.info()



# podstawowe czyszczenie
# jeśli w kolumnie 'Squad' są NaN albo jakieś „Total”, to odfiltruj
df_table = df_table[df_table['Squad'].notna()].copy()

# Poprawienie nazw drużyn
df_table['Squad'] = df_table['Squad'].str.extract(r'\[(.*?)\]', expand=False).fillna(df_table['Squad'])
df_table['Squad'] = df_table['Squad'].str.strip()

# Rzutowanie kolumn na typ numeryczny:
num_cols = ['Rk', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts', 'Pts/MP', 'Attendance']
for col in num_cols:
    df_table[col] = pd.to_numeric(df_table[col], errors='coerce')

# Sprawdzenie braków danych:
df_table.isna().sum()


# Połączenie kilku akruszy
# wczytanie dwóch arkuszy
std = pd.read_excel("../data/raw/Dane_Ekstraklasa.xlsx",
                    sheet_name="Squad Standard Stats",
                    engine="openpyxl",
                    skiprows=1)

shoot = pd.read_excel("../data/raw/Dane_Ekstraklasa.xlsx",
                      sheet_name="Squad Shooting",
                      engine="openpyxl",
                      skiprows=1)

# kolumna 'Squad' ma taką samą czystą postać jak w df_table
for df in [std, shoot]:
    df['Squad'] = df['Squad'].str.extract(r'\[(.*?)\]', expand=False).fillna(df['Squad'])
    df['Squad'] = df['Squad'].str.strip()

# tylko interesujące nas kolumny 
std_small = std[['Squad', '# Pl', 'Age', 'Poss', 'Gls', 'Ast', 'G+A']].rename(
    columns={'Gls': 'Gls_std', 'Ast': 'Ast_std', 'G+A': 'G+A_std'}
)
shoot_small = shoot[['Squad', 'Gls', 'Sh', 'SoT', 'Sh/90', 'SoT/90', 'G/Sh', 'G/SoT']].rename(
    columns={'Gls': 'Gls_shoot'}
)


# Merge po nazwie drużyny
df = df_table.merge(std_small, on='Squad', how='left', suffixes=('', '_std'))
df = df.merge(shoot_small, on='Squad', how='left', suffixes=('', '_shoot'))

df.head()


df.shape
df['Squad'].unique()
df.isna().sum()



df.to_csv(
    "../data/processed/ekstraklasa_clean.csv",
    index=False,
    encoding='utf-8'
)






   Rk           Squad  MP   W   D   L  GF  GA  GD  Pts  Pts/MP  Attendance  \
0   1     Lech Poznań  34  22   4   8  68  31  37   70    2.06      28.947   
1   2           Raków  34  20   9   5  51  23  28   69    2.03       5.427   
2   3     Jagiellonia  34  17  10   7  56  42  14   61    1.79      16.475   
3   4  Pogoń Szczecin  34  17   7  10  59  40  19   58    1.71      17.792   
4   5    Legia Warsaw  34  15   9  10  60  45  15   54    1.59      24.691   

                   Top Team Scorer           Goalkeeper  \
0                Mikael Ishak - 21       Bartosz Mrozek   
1        Jonatan Braut Brunes - 14     Kacper Trelowski   
2                  Jesús Imaz - 16  Sławomir Abramowicz   
3          Efthimis Koulouris - 28    Valentin Cojocaru   
4  Marc Gual, Bartosz Kapustka - 9       Kacper Tobiasz   

                                   Notes  
0   → Champions League via league finish  
1  → Conference League via league finish  
2  → Conference League via league finish  
3   